In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import requests
from itertools import chain

from bs4 import BeautifulSoup
import pandas as pd
import time

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
url = "https://www.pro-football-reference.com/years/"
options = Options()
options.headless = True
b = webdriver.Chrome(options=options)
b.get(url)

In [5]:
page = BeautifulSoup(b.page_source)
table = page.find(id = "div_years")
rows = []
table.find_all("th")
for th in table.find_all("th"):
    for data in th.find_all("a", href = True):
        row = [data.get_text(), data['href']]
        rows.append(row)

links = pd.DataFrame(rows, columns=['year', 'link'])
links

,year,link
0,2020,/years/2020/
1,2019,/years/2019/
2,2018,/years/2018/
3,2017,/years/2017/
4,2016,/years/2016/
...,...,...
96,1924,/years/1924/
97,1923,/years/1923/
98,1922,/years/1922/
99,1921,/years/1921_APFA/


In [ ]:
###### home_url = "https://www.pro-football-reference.com"
def get_data(link):
    url = home_url + link + 'passing.htm'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    tables = soup.find_all("table")
    
    table = tables[0]
    tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                            for row in table.find_all("tr")]
    df = pd.DataFrame(tab_data)
    df.columns = df.iloc[0,:]
    df.drop(index=0,inplace=True)
    df['year'] = link
    
    return df


df_19 = get_data('/years/2019/')
df_18 = get_data('/years/2018/')

data = pd.concat([
    df_19, 
    df_18
]).reset_index(drop=True)

data 

In [16]:
import time
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

url = "https://www.pro-football-reference.com/years/2018/"
driver.get(url)
time.sleep(3)
page = BeautifulSoup(driver.page_source)
table = page.find(id = 'team_stats')
tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                            for row in table.find_all("tr")]
df = pd.DataFrame(tab_data)
df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,,,,,Tot Yds & TO,,,Passing,Rushing,Penalties,...,None,None,None,None,None,None,None,None,None,None
1,Rk,Tm,G,PF,Yds,Ply,Y/P,TO,FL,1stD,...,Yds,TD,Y/A,1stD,Pen,Yds,1stPy,Sc%,TO%,EXP
2,1,Kansas City Chiefs,16,565,6810,996,6.8,18,6,384,...,1855,16,4.8,108,137,1152,37,52.6,10.5,298.14
3,2,Los Angeles Rams,16,527,6738,1060,6.4,19,7,401,...,2231,23,4.9,134,96,878,31,48.6,9.8,239.79
4,3,New Orleans Saints,16,504,6067,1010,6.0,16,9,377,...,2025,26,4.3,134,94,939,33,51.8,9.5,237.73
5,4,New England Patriots,16,436,6295,1073,5.9,18,7,365,...,2037,18,4.3,131,93,744,28,40.8,9.2,190.96
6,5,Indianapolis Colts,16,433,6179,1070,5.8,24,9,371,...,1718,13,4.2,96,120,953,38,41.2,13.2,140.47
7,6,Seattle Seahawks,16,428,5653,1012,5.6,11,4,329,...,2560,15,4.8,133,111,959,40,39.8,6.1,128.78
8,7,Los Angeles Chargers,16,428,5962,945,6.3,19,7,338,...,1873,16,4.7,100,113,967,25,42.4,10.0,170.19
9,8,Pittsburgh Steelers,16,428,6453,1058,6.1,26,9,378,...,1445,16,4.2,88,111,1026,39,36.8,14.3,156.16


In [2]:
# copied from scraper.py

class Sports_Ref_Scraper:
    def __init__(self, driver, home_url):
        self.driver = driver
        self.home_url = home_url
        self.get_links()
        
    def get_links(self):
        url = self.home_url + '/years/'
        self.driver.get(url)
        page = BeautifulSoup(self.driver.page_source)
        table = page.find(id = "div_years")
        rows = []
        table.find_all("th")
        for th in table.find_all("th"):
            for data in th.find_all("a", href = True):
                row = [data.get_text(), data['href']]
                rows.append(row)

        links = pd.DataFrame(rows, columns=['year', 'link'])
        self.links = links
    
    def get_passing_data(self, link):
        url = self.home_url + link + 'passing.htm'
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        tables = soup.find_all("table")

        table = tables[0]
        tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                                for row in table.find_all("tr")]
        df = pd.DataFrame(tab_data)
        df.columns = df.iloc[0,:]
        df.drop(index=0,inplace=True)
        df['year'] = link

        return df
    
    def giant_concat(self, collection):
        list_of_dicts = [cur_df.T.to_dict().values() for cur_df in collection]    
        giant_concat_df = pd.DataFrame(list(chain(*list_of_dicts)))
        giant_concat_df = giant_concat_df.replace(r'^\s*$', np.NaN, regex=True)
        return giant_concat_df
    
    def get_individual_passing(self, num_years, label):
        
        collection = []
        for i in range(0, num_years):
            link = self.links.loc[i]['link']
            df = self.get_passing_data(link)
            collection.append(df)
        
        combined = self.giant_concat(collection)
        combined.to_csv(label + '.csv')
    
    def get_team_stats_helper(self, link, idd):
        url = self.home_url + link
        self.driver.get(url)
        time.sleep(3)
        page = BeautifulSoup(self.driver.page_source)
        table = page.find(id = idd)
        tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                                    for row in table.find_all("tr")]

        df = pd.DataFrame(tab_data)
        df.columns = df.iloc[0,:]
        df.drop(index=0,inplace=True)
        df['year'] = link

        return df.reset_index(drop=True)
    
    def get_team_stats(self, num_years, label, idd):
        collection = []
        for i in range(0, num_years):
            print("iteration: ", i)
            link = self.links.loc[i]['link']
            df = self.get_team_stats_helper(link, idd)
            collection.append(df)
            
        combined = self.giant_concat(collection)
        combined.to_csv(label + '.csv')
        
        
            
        


In [3]:
url = "https://www.pro-football-reference.com"
options = Options()
options.headless = True
b = webdriver.Chrome(options=options)

scraper = Sports_Ref_Scraper(b, url)

In [18]:
scraper.get_individual_passing(10, 'test')

In [7]:
scraper.get_team_stats(2, 'test_team', 'team_stats')

iteration:  0
iteration:  1


In [4]:
scraper.get_team_stats(2, 'test_team_passing', 'passing')

iteration:  0
iteration:  1


In [5]:
df = pd.read_csv('test_team_passing.csv')
df

,Unnamed: 0,Rk,Tm,G,Cmp,Att,Cmp%,Yds,TD,TD%,...,Y/G,Rate,Sk,NY/A,ANY/A,Sk%,4QC,GWD,EXP,year
0,0,1,Dallas Cowboys,8,227,352,64.5,193,11,3.1,...,293.1,88.0,24,6.2,6.0,6.4,2,2,17.63,/years/2020/
1,1,2,Atlanta Falcons,8,211,317,66.6,123,12,3.8,...,292.4,97.3,17,7.0,7.2,5.1,NaN,NaN,94.86,/years/2020/
2,2,3,Kansas City Chiefs,8,196,291,67.4,67,21,7.2,...,286.4,114.6,11,7.6,8.8,3.6,1,1,124.27,/years/2020/
3,3,4,Tampa Bay Buccaneers,8,204,309,66.0,63,20,6.5,...,265.8,102.8,10,6.7,7.4,3.1,2,2,90.46,/years/2020/
4,4,5,Buffalo Bills,8,189,282,67.0,87,16,5.7,...,263.4,101.9,13,7.1,7.5,4.4,2,3,95.05,/years/2020/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,67,31,Pittsburgh Steelers,16,315,510,61.8,233,18,3.5,...,186.3,76.1,32,5.5,4.6,5.9,1,2,-6.50,/years/2019/
68,68,32,Washington Redskins,16,298,479,62.2,393,18,3.8,...,175.8,83.0,50,5.3,4.9,9.5,1,1,-45.16,/years/2019/
69,69,NaN,Avg Team,NaN,354.1,557.9,63.5,269.1,24.9,4.5,...,235.0,90.4,39.9,6.29,6.2,6.7,1.8,2.4,73.6,/years/2019/
70,70,NaN,League Total,NaN,11331,17853,63.5,8610,797,4.5,...,235.0,90.4,1276,6.29,6.2,6.7,58,77,NaN,/years/2019/


In [21]:
df = pd.read_csv('actual.csv')
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Tot Yds & TO,Tot Yds & TO.1,Tot Yds & TO.2,Unnamed: 8,Unnamed: 9,...,Rushing.1,Rushing.2,Rushing.3,Rushing.4,Penalties,Penalties.1,Penalties.2,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,Rk,Tm,G,PF,Yds,Ply,Y/P,TO,FL,1stD,...,Yds,TD,Y/A,1stD,Pen,Yds,1stPy,Sc%,TO%,EXP
1,1,Kansas City Chiefs,8,253,3284,522,6.3,5,4,189,...,993,7,4.5,55,48,409,14,51.3,6.3,123.44
2,2,Tampa Bay Buccaneers,8,247,2947,523,5.6,8,4,179,...,821,8,4.0,44,49,450,22,47.3,7.7,76.31
3,3,Seattle Seahawks,7,240,2901,451,6.4,7,1,171,...,878,6,5.0,52,37,277,12,46.8,8.9,102.39
4,4,Green Bay Packers,7,219,2762,449,6.2,3,1,162,...,902,6,4.7,44,39,364,16,50.0,4.3,105.65
